In [16]:
def init_seq(rows, cols):

    M = []

    for r in range(rows):
        arr = []
        for c in range(cols):
            arr.append(float(r * cols + c))
        M.append(arr)

    return M

# --------------------------------------- #

def print_matrix(M):

    rows = len(M)
    cols = len(M[-1])

    mat_str = "\n"
    for i in range(rows):
        mat_str += "|\t"
        for j in range(cols):
            mat_str += str(M[i][j]) + "\t"
            if j == cols - 1:
                mat_str += "|\n"
    print(mat_str)

# --------------------------------------- #

def init_rand(rows, cols):
    import random

    M = []
    for r in range(rows):
        arr = []
        for c in range(cols):
            arr.append(float(random.randrange(10)))
        M.append(arr)

    return M

In [134]:
r = 5
c = 5

M_a = init_rand(r, c)
print_matrix(M_a)

M_b = init_rand(r, c)
print_matrix(M_b)


|	8.0	8.0	7.0	7.0	3.0	|
|	6.0	9.0	7.0	4.0	6.0	|
|	7.0	3.0	4.0	5.0	6.0	|
|	9.0	7.0	2.0	8.0	2.0	|
|	9.0	0.0	4.0	2.0	9.0	|


|	1.0	3.0	9.0	0.0	6.0	|
|	7.0	4.0	3.0	5.0	7.0	|
|	5.0	2.0	8.0	6.0	8.0	|
|	1.0	6.0	4.0	4.0	1.0	|
|	2.0	2.0	9.0	9.0	6.0	|



In [135]:
import numpy as np
np.matmul(M_a, M_b)

array([[112., 118., 207., 137., 185.],
       [120., 104., 207., 157., 195.],
       [ 65.,  83., 178., 113., 136.],
       [ 80., 111., 168.,  97., 139.],
       [ 49.,  65., 202., 113., 142.]])

In [147]:
def mat_mul(M_a, M_b):
    rows_a = len(M_a)
    cols_a = len(M_a[-1])
    rows_b = len(M_b)
    cols_b = len(M_b[-1])

    if cols_a != rows_b:
        print("[ERROR] #columns A must be equal to #rows B.\n")
        sys.exit(-1)

    M_c = []
    for i in range(rows_a):
        arr = []
        for k in range(cols_b):
            sum = 0.0
            for j in range(cols_a):
                sum += float(M_a[i][j]) * float(M_b[j][k])
            arr.append(sum)
        M_c.append(arr)
    return M_c

# --------------------------------------- #

def mat_mul_transpose(M_a, M_b):
    rows_a = len(M_a)
    cols_a = len(M_a[-1])
    rows_b = len(M_b)
    cols_b = len(M_b[-1])

    if cols_a != rows_b:
        print("[ERROR] #columns A must be equal to #rows B.\n")
        sys.exit(-1)

    # Calculate the transpose
    M_b_Tr = []
    for i in range(rows_b):
        arr = []
        for j in range(cols_b):
            arr.append(M_b[j][i])
        M_b_Tr.append(arr)

    M_c = []
    for i in range(rows_a):
        arr = []
        for k in range(cols_b):
            sum = 0.0
            for j in range(cols_a):
                sum += float(M_a[i][j]) * float(M_b_Tr[k][j])
            arr.append(sum)
        M_c.append(arr)
    return M_c

# --------------------------------------- #

def _multi(M_a, M_b, M_c, cols_c, rows_c):
    cols_a = len(M_a[-1])
    cols_b = len(M_b[-1])

    aux = []
    for i in range(rows_c[0], rows_c[1]):
        arr = []
        for k in range(cols_c[0], cols_c[1]):
            sum = 0.0
            for j in range(cols_a):
                sum += float(M_a[i][j]) * float(M_b[j][k])
            arr.append(sum)
        aux.append(arr)

    # Store the values in the result matrix
    i = 0
    for j in range(rows_c[0], rows_c[1]):
        M_c[j] = aux[i]
        i += 1

def mat_mul_multithread(M_a, M_b):
    import sys
    import threading

    NUM_THREADS = 2
    rows_a = len(M_a)
    cols_a = len(M_a[-1])
    rows_b = len(M_b)
    cols_b = len(M_b[-1])

    if cols_a != rows_b:
        print("[ERROR] #columns A must be equal to #rows B.\n")
        sys.exit(-1)

    rows_per_thread = int(rows_a / NUM_THREADS)
    rest_of_matrix = rows_a % NUM_THREADS

    # print(rows_per_thread, rest_of_matrix)
    M_c = [[0.0] * cols_b] * rows_a

    # Create and start the threads
    threads = []
    results = [None] * NUM_THREADS
    for i in range(NUM_THREADS):
        # Calculate the params to pass them to the thread
        # TODO: The last thread operates the rest. Modify in a future and
        # TODO: let the first thread to end, operate the rest.
        cols_c_start = 0
        cols_c_end = cols_b
        rows_c_start = rows_per_thread * i
        rows_c_end = rows_c_start + rows_per_thread

        if (i == NUM_THREADS - 1) and (rest_of_matrix != 0):
            rows_c_end += rest_of_matrix

        cols_c = [cols_c_start, cols_c_end]
        rows_c = [rows_c_start, rows_c_end]

        t = threading.Thread(target=_multi, args=(M_a, M_b, M_c, cols_c, rows_c,))
        threads.append(t)
        t.start()

    # Join the threads
    for t in threads:
        t.join()

    return M_c

In [148]:
mat_mul_multithread(M_a, M_b)

[[112.0, 118.0, 207.0, 137.0, 185.0],
 [120.0, 104.0, 207.0, 157.0, 195.0],
 [65.0, 83.0, 178.0, 113.0, 136.0],
 [80.0, 111.0, 168.0, 97.0, 139.0],
 [49.0, 65.0, 202.0, 113.0, 142.0]]

In [138]:
mat_mul_transpose(M_a, M_b)

[[112.0, 118.0, 207.0, 137.0, 185.0],
 [120.0, 104.0, 207.0, 157.0, 195.0],
 [65.0, 83.0, 178.0, 113.0, 136.0],
 [80.0, 111.0, 168.0, 97.0, 139.0],
 [49.0, 65.0, 202.0, 113.0, 142.0]]

In [78]:
mat_mul(M_a, M_b)

[[26.0, 57.0], [5.0, 45.0]]